In [0]:
# Import the function from the src module
%run "/Workspace/Users/lucasmatborges@gmail.com/CaseBEESAmbev/Databricks/src/silver_function"

In [0]:
from datetime import datetime, UTC

ingest_date = datetime.now(UTC).strftime("%Y-%m-%d")
print("📅 Auto ingest date:", ingest_date)

In [0]:
# ---------------------------------------------------
# 1. Auto-generate ingest date = today in UTC
# ---------------------------------------------------
ingest_date = datetime.now(UTC).strftime("%Y-%m-%d")
print(f"📅 Processing Silver for date: {ingest_date}")

# ---------------------------------------------------
# 2. Bronze raw JSON Lines path
# ---------------------------------------------------
bronze_raw_file = f"/mnt/datalake/bronze/breweries/date={ingest_date}/raw.json"

# ---------------------------------------------------
# 3. Read Bronze JSON
# ---------------------------------------------------
df_bronze = spark.read.json(bronze_raw_file)

# ---------------------------------------------------
# 4. Transformations → Silver Curated
# ---------------------------------------------------
df_silver = transform_silver(df_bronze, ingest_date)

display(df_silver.limit(10))

# ---------------------------------------------------
# 5. Write Silver Delta (partitioned)
# ---------------------------------------------------
silver_path = "/mnt/datalake/silver/breweries"

(
    df_silver.write
        .format("delta")
        .mode("overwrite")
        .option("replaceWhere", f"ingest_date = '{ingest_date}'")
        .partitionBy("country", "state")
        .save(silver_path)
)

print("✔ Silver Delta written successfully!")
